# BNFO262: ChIP-Seq workshop

**Introduction:**

This workshop will walk you through an example of ChIP-seq analysis. We will focus on running tools as command lines and simple bash scripts – I recommend having a cheat-sheet like this [one](http://cheatsheetworld.com/programming/unix-linux-cheat-sheet/). 


We will use some common tools such as: 
* **bwa** for alignment (<a href="http://bio-bwa.sourceforge.net/bwa.shtml" target="_blank">http://bio-bwa.sourceforge.net/bwa.shtml</a>),
* **IGV** for visualization (<a href="http://software.broadinstitute.org/software/igv/home" target="_blank">http://software.broadinstitute.org/software/igv/home</a>) 
* Most of the workshop will be done using **HOMER** (e.g., QC of the data, peak calling etc.; <a href="http://homer.ucsd.edu/homer/" target="_blank">http://homer.ucsd.edu/homer/</a>). HOMER was created by Chris Benner at UCSD, and I love the documentation and tutorials and the threaded humor. To install HOMER follow <a href="https://github.com/biom262/cmm262-2020/blob/master/Module_5/Notebooks/Install_Homer.ipynb" target="_blank">these instructions</a>.

**Note:** Please use the 16 CPU DataHub environment when running this notebook. Otherwise, some of the commands may take a while to run.

To make it smoother, first type the command below in your terminal window.

In [ ]:
%%bash
source activate r-bio

During the workshop, and in general, it is always good to type the command and get the notes and use options of the command. Thus e.g., typing bwa would yield this output (capped after several lines):

```
(r-bio) agoren@dsmlp-jupyter-agoren$ bwa

Program: bwa (alignment via Burrows-Wheeler transformation)
Version: 0.7.17-r1188
Contact: Heng Li <lh3@sanger.ac.uk>

Usage:   bwa <command> [options]

Command: index         index sequences in the FASTA format
         mem           BWA-MEM algorithm
         fastmap       identify super-maximal exact matches
         pemerge       merge overlapping paired ends (EXPERIMENTAL)
         aln           gapped/ungapped alignment
         samse         generate alignment (single ended)
         sampe         generate alignment (paired ended)
         bwasw         BWA-SW for long queries

         shm           manage indices in shared memory
         fa2pac        convert FASTA to PAC format
         pac2bwt       generate BWT from PAC
         pac2bwtgen    alternative algorithm for generating BWT
         bwtupdate     update .bwt to the new format
         bwt2sa        generate SA from BWT and Occ

Note: To use BWA, you need to first index the genome with `bwa index'.
      There are three alignment algorithms in BWA: `mem', `bwasw', and
      `aln/samse/sampe'. If you are not sure which to use, try `bwa mem'
      first. Please `man ./bwa.1' for the manual.
```

## Part 0

### 1. Organize directories

Before we begin, we will create directories to organize our analysis:




In [ ]:
%%bash
# first, let's make sure we're in the right directory
cd ~/module-2-chipseq
mkdir chipseq_workshop
mkdir chipseq_workshop/aligned
mkdir chipseq_workshop/tagdirs

In [ ]:
# persist the change in directory to other cells in this notebook
%cd ~/module-2-chipseq

### 2. Generate symbolic links
Let's generate a couple [symbolic links](https://linuxize.com/post/how-to-create-symbolic-links-in-linux-using-the-ln-command/) to make it easier to type file paths. You can think of them as shortcuts.

```
ln -s path_to_file_or_directory symlink_name
```

In [ ]:
%%bash
ln -s ~/public/chipseq/fastqs/ chipseq_workshop/fastqs
ln -s ~/public/chipseq/mm9/ chipseq_workshop/mm9
ln -s ~/public/chipseq/mm9/mm9.fa.fai chipseq_workshop/fai

---

## Part I
We will start with FASTQ files and perform many of the basic analysis tasks that one might normally do when analyzing ChIP-seq data. 


### **1.** Align FASTQ reads using `bwa mem`.
The fastqs are at: `~/public/chipseq/fastqs`

But we made a symbolic link so we can access them at `chipseq_workshop/fastqs`

These files are originally from the following study investigating the roles that reprogramming factors play when transforming MEF (fibroblasts) into embryonic stem cells.
[Chronis et al. Cooperative Binding of Transcription Factors Orchestrates Reprogramming](https://www.ncbi.nlm.nih.gov/pubmed/28111071)
Sequencing Data: [GSE90893](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE90893)

For this tutorial we extracted the ChIP-seq experiments for several transcription factors and histone modifications performed on ESC (embryonic stem cells). To reduce runtimes, only reads that mapped to chr17 (and chr17_random) are included. (the random chromosomes are explained in this link – http://genome.ucsc.edu/FAQ/FAQdownloads#download10.)

**Question:** how would you generate such a file with only one chromosome?

To align the reads we will use bwa. It is always a good practice to look at the manual of each tool you use, so you get an idea of options and documentation.
Initially, to get an idea of the command we will run it on one file as follows:
(notice if you have very long lines of code you can break it up with a "\\")

In [ ]:
%%bash
# this command will take ~33 secs to run
bwa mem -t 16 chipseq_workshop/mm9/mm9.fa chipseq_workshop/fastqs/oct4-esc.chr17.2m.fastq \
| samtools sort -@ 16 -o chipseq_workshop/aligned/oct4-esc.chr17.2m.bam

It is a good practice to always double check datasets before you start analyzing them. For instance, use samtools to view the files.

In [ ]:
%%bash
samtools view chipseq_workshop/aligned/oct4-esc.chr17.2m.bam | head -n 10

We should also validate that the files are indeed what they should be (e.g., aligned to chr 17, and have 2M reads). 

To calculate the number of reads do:


In [ ]:
%%bash
samtools view chipseq_workshop/aligned/oct4-esc.chr17.2m.bam | wc -l

Alternatively, another option that is even better for here is to use samtools 

In [ ]:
%%bash
samtools flagstat chipseq_workshop/aligned/oct4-esc.chr17.2m.bam

If you want to understand better the way SAM files are organized you can follow <a href="https://samtools.github.io/hts-specs/SAMv1.pdf" target="_blank">https://samtools.github.io/hts-specs/SAMv1.pdf</a> section 1.4.


To do it properly, we will use a "for loop" to get BAMs from all the fastqs in the directory.

Importantly, always check that what you think you asked to be done in your code, is indeed what the machine understood.
echo is very useful - it should print out what the code is really saying. So first, let's make sure the files that are used in the for loop are the right ones:

In [ ]:
%%bash
for f in chipseq_workshop/fastqs/*fastq; do
    echo $f
done

note, if you want to view only a subset of the output (e.g., the directory has many files and you want to view only the first 3), you can just pipe the output to head or tail as follows:

In [ ]:
%%bash
for f in chipseq_workshop/fastqs/*fastq; do
    echo $f
done | head -n 3

Next, let's get the file name for each file, and strip it from the path and anything else, such as the extension. We will also check it using echo:

In [ ]:
%%bash
for f in chipseq_workshop/fastqs/*fastq; do
    fname="$(basename $f .fastq)"; 
    echo $fname
done

Let's add the command we used before, but again, echo it as well. Given that the > symbol would write to the file, we will use quotes for the echo: 

In [ ]:
%%bash
for f in chipseq_workshop/fastqs/*fastq; do
    fname="$(basename $f .fastq)";
    echo "bwa mem -t 16 chipseq_workshop/mm9/mm9.fa $f | samtools sort -@ 16 -o > chipseq_workshop/aligned/$fname.bam" ; 
done

Now, we can run it - need to make sure we removed the quotes and echo:

In [ ]:
%%bash
# in total, this should take about ~2.7 mins to run
for f in chipseq_workshop/fastqs/*fastq; do
    fname="$(basename $f .fastq)";
    bwa mem -t 16 chipseq_workshop/mm9/mm9.fa $f | samtools sort -@ 16 -o chipseq_workshop/aligned/$fname.bam;
done

This will produce BAM files for the 6 datasets. HOMER can analyze SAM files and if it receives BAM files it converts them to SAM so samtools has to be available (you can check that by typing “samtools” in the command line). 

--- 
### **2.** Create a “tag directory” 

**These commands should be run directly in the terminal**

For the example Oct4 ChIP-seq experiment using the makeTagDirectory command. Start by typing makeTagDirectory (without any options) in your command line, it will provide the usage, some info about the command and a full list of program options – as I mentioned above, I highly recommend doing that whenever you use a new tool and a new command. 

Tag directories are analogous to sorted bam files and are the starting point for most HOMER operations like finding peaks, creating visualization files, or calculating read densities. The command also performs several quality control and parameter estimation calculations. The command has the following form:  
    


```
makeTagDirectory <Output Tag Directory> [options] <input SAM/BAM file1> [input SAM/BAM file2] ...
``` 

To create a tag directory for the Oct4 experiment, open the terminal, make sure you are in the module-9-chipseq folder, and run the following command with recommended options:

In [ ]:
%%bash
# should take ~1 min
makeTagDirectory chipseq_workshop/tagdirs/oct4-esc -genome chipseq_workshop/mm9 -checkGC \
chipseq_workshop/aligned/oct4-esc.chr17.2m.bam

The command will take several seconds to run. What it is doing is parsing through the BAM file, removing reads that do not align to a unique position in the genome, separating reads by chromosome and sorting them by position, calculating how often reads appear in the same position to estimate the clonality (i.e. PCR duplication), calculating the relative distribution of reads relative to one another to estimate the ChIP-fragment length, calculating sequence properties and GC-content of the reads and performs a simple enrichment calculation to check if the experiment looks like a ChIP-seq experiment (vs. an RNA-seq experiment).

The command creates a new directory, in this case named **oct4-esc**. Inside the directory are several text files that contain various QC results. 

Try looking at the following using the "head" command:

> * **tagInfo.txt** - summary information from the experiment, including read totals.
> * **tagFreqUniq.txt** - nucleotide frequencies relative to the 5’ end of the sequencing reads.
> * **genomeGCcontent.txt** - distribution of ChIP-fragment GC%
> * **tagAutocorrelation.txt** - relative distribution of reads found on the same strand vs. different strands.
> * **tagCountDistribution.txt** - number of reads appearing at the same positions.


In [ ]:
# Use this box to look at the various files


--- 
### **3.** Create “tag directories” for all samples

By the following code, using a ‘for loop’, again pasting this command in the terminal.

In [ ]:
%%bash
# should take ~1 min
for f in chipseq_workshop/aligned/*.bam; do
    fname="$(basename $f .chr17.2m.bam)";
    nohup makeTagDirectory chipseq_workshop/tagdirs/"$fname" -genome chipseq_workshop/mm9 \
    -checkGC "$f" 2> chipseq_workshop/tagdirs/"$fname".out &
done

At this point you should have 6 tag directories. Look through the QC stats of the various ones.

---
### **4.** Next we will visualize the ChIP-seq experiments.

By creating bedGraph files from the tag directories and using the IGV genome browser to look at the results. We will do this using the makeUCSCfile command. For most ChIP-seq experiments all you need to do is specify the tag directory and specify “-o auto” for the command to automatically save the bedGraph file inside the tag directory:

```
makeUCSCfile chipseq_workshop/tagdirs -o auto
```

For a specific dataset, e.g. Oct4, the command would be:

```
makeUCSCfile chipseq_workshop/tagdirs/oct4-esc/ -o auto
```

This creates the file “oct4-esc/oct4-esc.ucsc.bedGraph.gz”. This file format specifies the normalized read depth at variable intervals along the genome (use zmore and the filename to view the file format for yourself). 

Now make these for all samples:

In [ ]:
%%bash
mkdir -p chipseq_workshop/viewer
for dir in chipseq_workshop/tagdirs/*-esc; do
    dirname="$(basename $dir)"
    nohup makeUCSCfile $dir -o chipseq_workshop/viewer/$dirname.ucsc.bedGraph \
    2> chipseq_workshop/viewer/$dirname.out &
done

### To view the file in the genome browser, do the following:

Download the files to your computer. To do this, we will first need to copy the files to the private directory as follows:


In [ ]:
%%bash
mkdir ~/private/viewer/
cp chipseq_workshop/viewer/*gz ~/private/viewer/

Now, you can run the following command from the terminal of your local computer:
```bash
scp -r 'USERNAME@dsmlp-login.ucsd.edu:~/viewer/*gz' ./
```

_Note_: You must have `ssh` installed locally for this to work. Make sure to replace USERNAME with your UCSD AD username. Provide your AD password when prompted.

**Open IGV.** Make sure you use the right genome (mm9; it is a good practice to have!) and drag the file to the center window (or select file -> load from file).

The read pileups will display the relative density of ChIP-seq reads at each position in the genome. We only have data for chr17 in this example, so we can stick to that chromosome.

---
### **5.** See if there are any interesting patterns in the data that catch your eye.

Try visiting the Pou5f1 locus (the gene for Oct4) by typing the gene name into the search bar at the top. Once at the Pou5f1 locus, zoom out (alt+click or scale on top right) to see if there any nearby sites that might resemble enhancers.

Each dataset was created by a different antibody, and they can be divided into three types: TFs, HMs and global input. Since we will need to treat each type differently, I recommend making a directory for each – input, tfs and hms and move the tag directories to the relevant one (e.g. tfs/oct4-esc/, etc). 
